# Titanic for beginners
it is basic introduction to Kaggle.

## Workflow
1. Import Necessary Libraries
2. Acquire training and testing data.
3. Analyze, Visualize data
  1. Outlets (errors or possibly innacurate values) ?
  2. Create new feature?
4. Clearning data
5. Choosing the Best Model
6. Creating Submission File

## 1. Import Necessary Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import subprocess

%matplotlib inline

## 2. Acquire training and testing data

In [2]:
# check input files
print(subprocess.check_output(["ls", "../input"]).decode("utf8"))

gender_submission.csv
test.csv
train.csv

